In [1]:
import os

import pandas as pd

from bert_deid import model as bert_deid_model
from pydeid import annotation

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Load in model

In [2]:
# Load a trained model
bert_model = bert_deid_model.BertForDEID(
    model_dir="/db/git/bert-deid/models/i2b2_2014"
)
# bert_model.to('cpu')
# bert_model.eval()

Loading model and configuration from /db/git/bert-deid/models/i2b2_2014.


07/26/2019 13:27:37 - INFO - bert_deid.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/alistairewj/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


### Load in text files

In [4]:
base_path = 'tests/fake-data/radiology-reports/txt'
reports_list = os.listdir(base_path)
reports_list.sort()

reports = {}
for f in reports_list:
    with open(os.path.join(base_path, f), 'r') as fp:
        reports[f] = ''.join(fp.readlines())

# BERT deid

### Run bert-deid

In [5]:
anns_bert = {}
for f in reports_list:
    print(f'De-identifying {f}')
    
    text = reports[f]
    
    # ann with bert
    ann = bert_model.annotate(
        text, document_id=f)

    # merge intervals that are close together
    ann = annotation.merge_intervals(
        ann, dist=1, text=text)

    # post-fix to reduce false positives
    ann = bert_model.postfix(ann, text)
    
    # add to dict
    anns_bert[f] = ann
    
    print('First 5 rows of deid:')
    display(ann.head())
    

De-identifying 500.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
23,500.txt,bert.1.85,bert-base-cased,528,530,70,AGE,None,9.473902
21,500.txt,bert.0.53,bert-base-cased,339,344,-0500,CONTACT,None,5.124740
49,500.txt,bert.7.57,bert-base-cased,2071,2076,-0500,CONTACT,None,5.286543
55,500.txt,bert.9.22,bert-base-cased,2408,2422,(617) 555-1234,CONTACT,None,6.621456
5,500.txt,bert.0.15,bert-base-cased,143,151,01/02/40,DATE,None,10.202400


De-identifying 501.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
21,501.txt,bert.2.10,bert-base-cased,530,532,96,AGE,None,9.324322
22,501.txt,bert.3.50,bert-base-cased,801,803,96,AGE,None,8.499738
27,501.txt,bert.5.31,bert-base-cased,1549,1563,(617) 555-1234,CONTACT,None,7.454946
6,501.txt,bert.0.14,bert-base-cased,143,151,14/02/13,DATE,None,10.328542
14,501.txt,bert.0.30,bert-base-cased,247,255,01/05/10,DATE,None,10.332378


De-identifying 502.txt
First 5 rows of deid:


...

,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
19,504.txt,bert.1.74,bert-base-cased,489,491,74,AGE,None,9.475336
21,504.txt,bert.3.11,bert-base-cased,779,781,74,AGE,None,8.938679
16,504.txt,bert.0.49,bert-base-cased,337,345,000-0504,CONTACT,None,4.797463
35,504.txt,bert.5.33,bert-base-cased,1743,1757,(617) 555-1234,CONTACT,None,6.334353
6,504.txt,bert.0.15,bert-base-cased,143,151,06/26/36,DATE,None,10.342719


De-identifying 505.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
25,505.txt,bert.2.3,bert-base-cased,553,555,62,AGE,None,9.540651
22,505.txt,bert.0.50,bert-base-cased,340,345,-0505,CONTACT,None,6.708478
34,505.txt,bert.5.38,bert-base-cased,1729,1743,(617) 555-1234,CONTACT,None,6.706555
6,505.txt,bert.0.14,bert-base-cased,143,151,09/15/47,DATE,None,10.788244
16,505.txt,bert.0.32,bert-base-cased,247,255,07/19/10,DATE,None,10.313417


De-identifying 506.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
22,506.txt,bert.1.71,bert-base-cased,529,531,29,AGE,None,8.654554
25,506.txt,bert.3.2,bert-base-cased,860,862,29,AGE,None,8.835622
19,506.txt,bert.0.45,bert-base-cased,337,345,000-0506,CONTACT,None,6.489229
40,506.txt,bert.4.36,bert-base-cased,1449,1463,(617) 632-7234,CONTACT,None,5.441032
5,506.txt,bert.0.14,bert-base-cased,143,151,01/01/60,DATE,None,9.933118


### (Optional) Annotate manual text

In [7]:
text = 'Patient is a 64 yo man with a serious condition.'

# ann with bert
df = bert_model.annotate(text, document_id='0')

# merge intervals that are close together
df = annotation.merge_intervals(df, dist=1, text=text)

# post-fix to reduce false positives
df = bert_model.postfix(df, text)

display(df)

,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,0,bert.0.3,bert-base-cased,13,15,64,AGE,None,9.272903


# pydeid

In [8]:
from pydeid import annotator, metrics

modules = ['age', 'name', 'date',
           'location', 'telephone', 'initials',
           'rr',
           'crossreference']

# initialize deid module
mdl = annotator.Pattern(modules=modules)

In [9]:
anns_pydeid = {}
for f in reports_list:
    print(f'De-identifying {f}')
    text = reports[f]

    ann = mdl.annotate(text, document_id=f)
    
    anns_pydeid[f] = ann
    
    print('First 5 rows of deid:')
    display(ann.head())

De-identifying 500.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,500.txt,None,regex_name_pattern_a00,18,22,BETH,Name,None,NaN
1,500.txt,None,regex_name_pattern_a00,23,29,ISRAEL,Name,None,NaN
2,500.txt,None,regex_name_pattern_a00,110,118,MARGARET,Name,None,NaN
3,500.txt,None,regex_name_pattern_a00,195,200,BAYES,Name,None,NaN
4,500.txt,None,regex_name_pattern_a00,201,209,GIOVANNI,Name,None,NaN


De-identifying 501.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,501.txt,None,regex_name_pattern_a00,18,22,BETH,Name,None,None
1,501.txt,None,regex_name_pattern_a00,23,29,ISRAEL,Name,None,None
2,501.txt,None,regex_name_pattern_a00,108,114,AURORA,Name,None,None
3,501.txt,None,regex_name_pattern_a00,220,225,TRAUM,Name,None,None
4,501.txt,None,regex_name_pattern_a00,1394,1400,HESTER,Name,None,None


De-identifying 502.txt
First 5 rows of deid:


...

,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,504.txt,None,regex_name_pattern_a00,18,22,BETH,Name,None,None
1,504.txt,None,regex_name_pattern_a00,23,29,ISRAEL,Name,None,None
2,504.txt,None,regex_name_pattern_a00,846,850,June,Name,None,None
3,504.txt,None,regex_name_pattern_a00,1556,1561,Franz,Name,None,None
4,504.txt,None,regex_name_pattern_a00,1699,1707,FREDERIC,Name,None,None


De-identifying 505.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,505.txt,None,regex_name_pattern_a00,18,22,BETH,Name,None,None
1,505.txt,None,regex_name_pattern_a00,23,29,ISRAEL,Name,None,None
2,505.txt,None,regex_name_pattern_a00,109,115,FUMIKO,Name,None,None
3,505.txt,None,regex_name_pattern_a00,1574,1578,ANNA,Name,None,None
4,505.txt,None,regex_name_pattern_a1,102,108,KARTIA,Name,None,None


De-identifying 506.txt
First 5 rows of deid:


,document_id,annotation_id,annotator,start,stop,entity,entity_type,comment,confidence
0,506.txt,None,regex_name_pattern_a00,18,22,BETH,Name,None,None
1,506.txt,None,regex_name_pattern_a00,23,29,ISRAEL,Name,None,None
2,506.txt,None,regex_name_pattern_a00,102,109,SHEPARD,Name,None,None
3,506.txt,None,regex_name_pattern_a00,110,114,JOHN,Name,None,None
4,506.txt,None,regex_name_pattern_a00,193,197,LINA,Name,None,None


# Concatenate bert/pydeid output and write out to file

In [11]:
for f in reports_list:
    ann = pd.concat([anns_bert[f], anns_pydeid[f]], ignore_index=True)
    
    # get output filename
    out_fn = f
    if out_fn[-4:] == '.txt':
        out_fn = out_fn[:-4]
    out_fn += '.gs'
    
    out_fn = os.path.join('tests', 'fake-data', 'radiology-reports', out_fn)
    print(f'Outputting to {out_fn}')
    ann.to_csv(out_fn, index=False)

Outputting to tests/radiology-reports/500.gs


FileNotFoundError: [Errno 2] No such file or directory: 'tests/radiology-reports/500.gs'